In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType, StructType, StructField, StringType
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import numpy as np
import os
from consts import QUESTIONS_PATH, JOBS_PATH, open_csv_file

# Initialize Spark session
spark = SparkSession.builder.appName("InterviewQuestionSelector").getOrCreate()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [0]:
# Load datasets into Spark DataFrames
job_postings = open_csv_file(spark, JOBS_PATH, 'all_jobpostings.csv')
code_questions = open_csv_file(spark, QUESTIONS_PATH, 'all_code_problems_with_solutions.csv')
open_questions = open_csv_file(spark, QUESTIONS_PATH, 'all_open_questions.csv')

# Preprocessing function to handle missing values and ensure string type
def preprocess_column_spark(df, column):
    df = df.withColumn(column, col(column).cast("string"))
    df = df.fillna({column: ""})
    return df

# Preprocess columns in the datasets
job_postings = preprocess_column_spark(job_postings, 'job_summary')
code_questions = preprocess_column_spark(code_questions, 'topics')
open_questions = preprocess_column_spark(open_questions, 'question')

In [0]:
pip install -q -U google-generativeai

## Job postings: filling in missing skills

In [0]:
from pyspark.sql.functions import udf, concat_ws
from pyspark.sql.types import ArrayType, StringType
import ast

# Define a UDF to safely parse the string to a list
def parse_skills(skills_str):
    try:
        return ast.literal_eval(skills_str)
    except (ValueError, SyntaxError):
        return []

parse_skills_udf = udf(parse_skills, ArrayType(StringType()))

# Apply the UDF to create a proper list column
job_postings = job_postings.withColumn("skills_list", parse_skills_udf("skills"))

# Convert the skills list to a single string
job_postings = job_postings.withColumn("skills_string", concat_ws(", ", "skills_list")) \
    .drop("skills", "skills_list").withColumnRenamed("skills_string", "skills")
job_postings.display()

In [0]:
import pandas as pd

# Convert dataset to pandas
job_postings_pandas = job_postings.toPandas()
empty_skills_count = job_postings_pandas[job_postings_pandas['skills'] == ''].shape[0]
print(empty_skills_count)

In [0]:
import google.generativeai as genai
import os
import time
from api_keys import API_KEYS

def infer_skills(job_summary):
    """
    Extracts skills from a job description using the Gemini model.

    Args:
        job_summary: The job description text.

    Returns:
        A comma-separated string of skills extracted from the job description.
    """

    if pd.isna(job_summary) or str(job_summary).strip() == '':
        return ''

    prompt = f"Infer a comma-separated list of skills required for the following job description:\n{job_summary}"

    try:
        response = model.generate_content(prompt)
        print("response:", response.text)
        return response.text.strip()
    except Exception as e:
        print(f"Error inferring skills: {e}")
        return ''
    
start_time = time.time()
running_time = 0
empty_skills_rows = job_postings_pandas[(job_postings_pandas['skills'] == '') & (job_postings_pandas['job_summary'].str.strip() != '')]

while (running_time < 3600) and (empty_skills_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "skills" column is empty
        empty_skills_rows = job_postings_pandas[(job_postings_pandas['skills'] == '') & (job_postings_pandas['job_summary'].str.strip() != '')]
        if empty_skills_rows.shape[0] == 0:
            break

        # Get the indices of the first 15 rows with empty "skills"
        indices_to_update = empty_skills_rows.index[:15]

        # Apply the UDF only to the selected rows
        job_postings_pandas.loc[indices_to_update, 'skills'] = (
            job_postings_pandas.loc[indices_to_update, 'job_summary']
                .apply(infer_skills)
        )
    running_time = time.time() - start_time

In [0]:
job_postings_pandas['skills'] = job_postings_pandas['skills'].fillna('')
empty_skills_count = job_postings_pandas[job_postings_pandas['skills'] == ''].shape[0]
job_postings_pandas['job_summary'] = job_postings_pandas['job_summary'].fillna('')
empty_job_summaries = job_postings_pandas[job_postings_pandas['job_summary'].str.strip() == ''].shape[0]
print("empty strings:", empty_skills_count)
print("empty job summaries:", empty_job_summaries)

In [0]:
from consts import JOBS_PATH
import os

job_postings_with_skills = spark.createDataFrame(job_postings_pandas)
job_postings_pandas.to_csv(os.path.join(JOBS_PATH, 'all_jobpostings_with_skills.csv'), index=False)

## Code questions: filling in missing topics

In [0]:
from pyspark.sql.functions import regexp_replace, concat_ws, split, col, expr

# Format the topics column to be a string containing comma-separated topics.
code_questions = code_questions.withColumn("topics_array", split(col("topics"), ", ")) \
    .withColumn("topics_array_cleaned", expr("transform(topics_array, x -> regexp_replace(x, \"'\", \"\"))")) \
    .withColumn("topics_formatted", concat_ws(", ", col("topics_array_cleaned"))) \
    .drop("topics_array", "topics_array_cleaned", "topics").withColumnRenamed("topics_formatted", "topics")

code_questions_pandas = code_questions.toPandas()

In [0]:
import google.generativeai as genai
import os
import time
from api_keys import API_KEYS

def extract_topics_from_question(question):
   if pd.isna(question) or question.strip() == '':
       return ''
   
   prompt = f"Analyze the following question and identify the specific skills being tested or evaluated. Return the skills as a comma-separated list of skills. If the question does not test any skills, return an empty string. Question: {question}"
   
   try:
       response = model.generate_content(prompt)
       skills = response.text.strip()
       print("response:", skills)
       return skills if skills else ''
   except Exception as e:
       print(f"Error extracting topics: {e}")
       return ''
   

start_time = time.time()
running_time = 0
empty_topics_rows = code_questions_pandas[(code_questions_pandas['topics'] == '') & (code_questions_pandas['question'].str.strip() != '')]

while (running_time < 900) and (empty_topics_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Filter the rows where the "topics" column is empty
        empty_topics_rows = code_questions_pandas[(code_questions_pandas['topics'] == '') & (code_questions_pandas['question'].str.strip() != '')]
        if empty_topics_rows.shape[0] == 0:
            break

        # Get the indices of the first 15 rows with empty "topics"
        indices_to_update = empty_topics_rows.index[:15]

        # Apply the UDF only to the selected rows
        code_questions_pandas.loc[indices_to_update, 'topics'] = (
            code_questions_pandas.loc[indices_to_update, 'question']
                .apply(extract_topics_from_question)
        )
    running_time = time.time() - start_time

In [0]:
empty_topics_count = code_questions_pandas[code_questions_pandas['topics'].isna()].shape[0]
print("nulls:", empty_topics_count)
code_questions_pandas['topics'] = code_questions_pandas['topics'].fillna('')
empty_topics_count = code_questions_pandas[code_questions_pandas['topics'] == ''].shape[0]
code_questions_pandas['question'] = code_questions_pandas['question'].fillna('')
empty_questions = code_questions_pandas[code_questions_pandas['question'].str.strip() == ''].shape[0]
print("empty strings:", empty_topics_count)
print("empty questions:", empty_questions)

In [0]:
from consts import QUESTIONS_PATH
import os

code_questions_with_topics = spark.createDataFrame(code_questions_pandas)
code_questions_pandas.to_csv(os.path.join(QUESTIONS_PATH, 'all_code_questions_with_topics.csv'), index=False)

In [0]:
code_questions_with_topics.display()

## Open questions: filling in missing topics

In [0]:
import pandas as pd

# Convert dataset to pandas
open_questions_pandas = open_questions.toPandas()
open_questions_pandas['topics'] = ''

In [0]:
import google.generativeai as genai
import os
from api_keys import API_KEYS
   
start_time = time.time()
running_time = 0
empty_topics_rows = open_questions_pandas[(open_questions_pandas['topics'] == '') & (open_questions_pandas['question'].str.strip() != '')]

while (running_time < 900) and (empty_topics_rows.shape[0] > 0):
    for api_key in API_KEYS.values():
        # Configure Gemini API
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        model = genai.GenerativeModel('gemini-1.5-flash')

        empty_topics_rows = open_questions_pandas[(open_questions_pandas['topics'] == '') & (open_questions_pandas['question'].str.strip() != '')]
        if empty_topics_rows.shape[0] == 0:
            break

        # Get the indices of the first 15 rows with empty "topics"
        indices_to_update = empty_topics_rows.index[:15]

        # Apply the UDF only to the selected rows
        open_questions_pandas.loc[indices_to_update, 'topics'] = (
            open_questions_pandas.loc[indices_to_update, 'question']
                .apply(extract_topics_from_question)
        )
    running_time = time.time() - start_time

In [0]:
from consts import QUESTIONS_PATH
import os

open_questions_with_topics = spark.createDataFrame(open_questions_pandas)
open_questions_pandas.to_csv(os.path.join(QUESTIONS_PATH, 'all_open_questions_with_topics.csv'), index=False)

In [0]:
open_questions_with_topics.display()

## Topics & skills embeddings

In [0]:
import pandas as pd

job_postings_with_skills = pd.read_csv(os.path.join(JOBS_PATH, "all_jobpostings_with_skills.csv"))
job_postings_with_skills['skills'] = job_postings_with_skills['skills'].fillna('')
code_questions_with_topics = pd.read_csv(os.path.join(QUESTIONS_PATH, "all_code_questions_with_topics.csv"))
open_questions_with_topics = pd.read_csv(os.path.join(QUESTIONS_PATH, "all_open_questions_with_topics.csv"))

In [0]:
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the model for embedding generation
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model for semantic similarity

# Tokenization and Embedding Generation
def preprocess_and_embed_list(text_column):
    """Tokenizes the list and generates embeddings for individual items."""
    # Split the list into individual items
    tokens = text_column.str.split(',').apply(lambda x: [t.strip().lower() for t in x])
    
    # Embed each item and aggregate embeddings
    embeddings = tokens.apply(lambda x: model.encode(x))
    aggregated_embeddings = embeddings.apply(lambda x: x.mean(axis=0))  # Mean pooling
    
    return tokens, aggregated_embeddings

job_postings_with_skills['skills_tokens'], job_postings_with_skills['skills_embeddings'] = preprocess_and_embed_list(job_postings_with_skills['skills'])
code_questions_with_topics['topics_tokens'], code_questions_with_topics['topics_embeddings'] = preprocess_and_embed_list(code_questions_with_topics['topics'])
open_questions_with_topics['topics_tokens'], open_questions_with_topics['topics_embeddings'] = preprocess_and_embed_list(open_questions_with_topics['topics'])

# Cartesian Product
cartesian_pairs_code = pd.DataFrame(
    itertools.product(job_postings_with_skills.index, code_questions_with_topics.index),
    columns=['job_idx', 'question_idx']
)
cartesian_pairs_open = pd.DataFrame(
    itertools.product(job_postings_with_skills.index, open_questions_with_topics.index),
    columns=['job_idx', 'question_idx']
)


In [0]:
display(cartesian_pairs_code)
display(cartesian_pairs_open)

In [0]:
def compute_match_score(row, questions_df, skills_column, topics_column):
    """
    Calculates the match score for a job-question pair.
    
    Parameters:
        - row: The row from the Cartesian product DataFrame.
        - questions_df: The DataFrame containing the questions (either 'code_questions' or 'open_questions').
        - skills_column: The column in the job postings DataFrame containing skill embeddings.
        - topics_column: The column in the questions DataFrame containing topic embeddings.
    
    Returns:
        - similarity: Cosine similarity between job skills and question topics.
    """
    # Get job skills and question topics embeddings
    job_skills = job_postings_with_skills.loc[row['job_idx'], skills_column]
    question_topics = questions_df.loc[row['question_idx'], topics_column]
    
    # Ensure embeddings are reshaped to 2D arrays
    job_skills = job_skills.reshape(1, -1) if len(job_skills.shape) == 1 else job_skills
    question_topics = question_topics.reshape(1, -1) if len(question_topics.shape) == 1 else question_topics
    
    # Compute cosine similarity
    similarity = cosine_similarity(job_skills, question_topics)
    return similarity[0][0]

# Apply the function to compute scores for code_questions
cartesian_pairs_code['skills_topics_score'] = cartesian_pairs_code.apply(
    compute_match_score, axis=1, questions_df=code_questions_with_topics, 
    skills_column='skills_embeddings', topics_column='topics_embeddings'
)

# Apply the function to compute scores for open_questions
cartesian_pairs_open['skills_topics_score'] = cartesian_pairs_open.apply(
    compute_match_score, axis=1, questions_df=open_questions_with_topics, 
    skills_column='skills_embeddings', topics_column='topics_embeddings'
)

#### Another try:

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Start Spark session
spark = SparkSession.builder.appName("JobQuestionMatching").getOrCreate()

# Load datasets into Spark DataFrames
job_postings_df = open_csv_file(spark, JOBS_PATH, "all_jobpostings_with_skills.csv")
job_postings_with_skills['skills'] = job_postings_with_skills['skills'].fillna('')
code_questions_df = open_csv_file(spark, QUESTIONS_PATH, "all_code_questions_with_topics.csv")
open_questions_df = open_csv_file(spark, QUESTIONS_PATH, "all_open_questions_with_topics.csv")

# Explode the skills and topics columns
job_postings_exploded = job_postings_df.withColumn("skill", explode(split("skills", ",")))
code_questions_exploded = code_questions_df.withColumn("topic", explode(split("topics", ",")))
open_questions_exploded = open_questions_df.withColumn("topic", explode(split("topics", ",")))

# Cartesian product between job postings and questions
cartesian_code = job_postings_exploded.crossJoin(code_questions_exploded)
cartesian_open = job_postings_exploded.crossJoin(open_questions_exploded)


In [0]:
# Convert to Pandas for embedding and similarity calculation
cartesian_code_pandas = cartesian_code.toPandas()
cartesian_open_pandas = cartesian_open.toPandas()

# Generate embeddings and compute similarity in Pandas
cartesian_code_pandas['skill_embedding'] = cartesian_code_pandas['skill'].apply(model.encode)
cartesian_code_pandas['topic_embedding'] = cartesian_code_pandas['topic'].apply(model.encode)

cartesian_code_pandas['similarity'] = cartesian_code_pandas.apply(
    lambda row: cosine_similarity([row['skill_embedding']], [row['topic_embedding']])[0][0], axis=1
)

In [0]:
# Convert back to Spark DataFrame for aggregation
similarity_df = spark.createDataFrame(cartesian_code_pandas)

# Aggregate similarity scores
aggregated_scores = similarity_df.groupBy("job_idx", "question_idx").sum("similarity")

## Heuristic to match questions to jobs
Questions with the highest hueristic grades wil be the most likely to appear in the interview.

In [0]:
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import FloatType
from scipy.spatial.distance import cosine

# Each question will be initially considered for each job posting.
jobs_with_code_questions = job_postings.crossJoin(code_questions)
jobs_with_open_questions = job_postings.crossJoin(open_questions)

# Map difficulty levels to numeric values
difficulty_map = {"Easy": 0, "Medium": 1, "Hard": 2}

# Change difficulty column to numeric
code_questions = code_questions.withColumn(
    "difficulty", col("difficulty").map(difficulty_map)
)

# Match question's difficulty to job posting's level
jobs_with_code_questions = jobs_with_code_questions.withColumn(
    "difficulty_match",
    1 - abs(col("difficulty") - col("level")) / 2
)

# Calculate cosine similarity between embeddings
def calculate_similarity(embedding1, embedding2):
    if embedding1 is None or embedding2 is None:
        return 0.0
    return 1 - cosine(embedding1, embedding2)
similarity_udf = udf(calculate_similarity, FloatType())

# Add topic similarity scores
jobs_with_code_questions = jobs_with_code_questions.withColumn(
    "emb_similarity",
    similarity_udf(col("jobposting_embedding", "topics_embedding"))
)
jobs_with_open_questions = jobs_with_open_questions.withColumn(
    "emb_similarity",
    similarity_udf(col("jobposting_embedding", "question_embedding"))
)

# Normalize Acceptance for code questions
max_acceptance = code_questions.agg({"acceptance": "max"}).collect()[0][0]
jobs_with_code_questions = jobs_with_code_questions.withColumn(
    "normalized_acceptance", col("acceptance") / max_acceptance
)

# Calculate Heuristic Score
def calculate_score(difficulty, similarity, acceptance):
    return 0.3 * difficulty + 0.5 * similarity + 0.2 * acceptance
calculate_score_udf = udf(calculate_score, FloatType())

jobs_with_code_questions = jobs_with_code_questions.withColumn(
    "heuristic_score",
    calculate_score_udf(
        col("difficulty_match"),
        col("emb_similarity"),
        col("normalized_acceptance"),
    ),
)
display(jobs_with_code_questions)

jobs_with_open_questions = jobs_with_open_questions.withColumn(
    "heuristic_score",
    calculate_score_udf(
        col("difficulty_match"),  # This is 0 for open questions
        col("emb_similarity"),
        lit(0)  # No acceptance column in open questions
    ),
)
display(jobs_with_open_questions)

In [0]:
# Select Top Questions for Each Job
from pyspark.sql import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy("job_id").orderBy(col("heuristic_score").desc())

top_code_questions = jobs_with_code_questions.withColumn(
    "rank", row_number().over(window_spec)
).filter(col("rank") <= 10)

top_open_questions = jobs_with_open_questions.withColumn(
    "rank", row_number().over(window_spec)
).filter(col("rank") <= 10)

display(top_code_questions)
display(top_open_questions)

In [0]:
# Join scores back to original DataFrames for easy analysis
result = cartesian_pairs.merge(job_postings_with_skills, left_on='job_idx', right_index=True)
result = result.merge(code_questions_with_topics, left_on='question_idx', right_index=True)

# Sort by match score
result = result.sort_values(by='heuristic_score', ascending=False)